In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif, SelectFdr, SelectKBest, chi2

import utils.dev_config as dev_conf
import utils.preprocessing as prep

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
def min_max_norm(x, a, b):
    x_min = x.min()
    x_max = x.max()
    return ((x - x_min) * (b - a)) / (x_max - x_min) + a

In [4]:
i = 0

In [5]:
# matrisome_df = pd.read_csv(matrisome_list, sep='\t')
matrisome_df = prep.load_matrisome_df(matrisome_list)
sig_deg_df = pd.read_csv(f"{dirs.analysis_dir}/{unified_dsets[i]}_sig_DESeq_results_xref_matrisome.tsv", sep = '\t')
matrisome_sig_deg_df = (
    sig_deg_df.query("in_matrisome == True")
        .reset_index(drop=True)
)

# Load and filter survival data

In [6]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["age_at_diagnosis", "bmi", "race", "ethnicity"]
dep_cols = ["vital_status", "survival_time"]
cat_cols = ["race", "ethnicity"]

In [7]:
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[i]}/survival_data.tsv", event_code)

In [8]:
filtered_survival_df = (
    survival_df[["sample_name"] + dep_cols + covariate_cols]
        .query("vital_status == 1")
        .dropna()
        .reset_index(drop=True)
        .pipe(pd.get_dummies, columns=cat_cols)
)

In [9]:
print(filtered_survival_df.shape[0])
print(filtered_survival_df.shape[0] / survival_df.shape[0])

48
0.18532818532818532


# Load normalized matrisome count data

In [10]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[i]}/norm_matrisome_counts.tsv", sep='\t')
norm_matrisome_survival_counts_t_df = (
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)]
        .set_index("geneID")                        # set as index so will be column names
        .transpose()
        .rename_axis(None, axis=1)                  # column.name will be set to "geneID", we don't want this
        .reset_index()                              # "sample_name" should now be its own column
        .rename({"index": "sample_name"}, axis=1)
)

In [11]:
joined_survival_counts_df = (
    pd.merge(filtered_survival_df, norm_matrisome_survival_counts_t_df, on="sample_name")
        .drop("vital_status", axis=1)
        .set_index("sample_name")
)
# # This feature is integral
# joined_survival_counts_df.age_at_diagnosis = joined_survival_counts_df.age_at_diagnosis.astype("int")

# Drop columns where min = max, these will provide no information

In [12]:
rep_val_cols = np.where(joined_survival_counts_df.min() == joined_survival_counts_df.max())[0]

In [13]:
filtered_joined_survival_counts_df = joined_survival_counts_df.drop(joined_survival_counts_df.columns[rep_val_cols], axis=1)
filtered_joined_survival_counts_df.shape

(48, 1004)

# Min-max normalize data

In [15]:
mm_norm_df = filtered_joined_survival_counts_df.apply(lambda x: min_max_norm(x, 0, 1), axis=0)

# Examine mutual information

In [16]:
# # The one-hot columns are the only remaining discrete variables
# discr_cols = np.where(filtered_joined_survival_counts_df.columns.str.contains("race|ethnicity"))[0]

# # The first column is the response variable, so we shift these indices
# discr_cols -= 1

In [17]:
X = joined_survival_counts_df.iloc[:, 1:].values
y = joined_survival_counts_df.iloc[:, 0].values
# X_mm = mm_norm_df.iloc[:, 1:].values
# y_mm = mm_norm_df.iloc[:, 0].values

In [91]:
# only include genes, since MI scores are independent
mi_res = mutual_info_regression(X[:, 10:], y, discrete_features=False)
mi_df = pd.DataFrame({"geneID": joined_survival_counts_df.columns[11:], "MI": mi_res})
mi_df = mi_df.sort_values("MI", ascending=False).reset_index(drop=True)
dep_mi_df = mi_df[mi_df.MI > 0]
dep_mi_df.shape

(491, 2)

In [84]:
coxph_df = pd.read_csv(f"{dirs.analysis_dir}/{unified_dsets[i]}_coxph_results.tsv", sep='\t')

In [85]:
sig_coxph_df = coxph_res_df[coxph_df.gene_pval < 0.05]

In [86]:
sig_coxph_genes = set(sig_coxph_df.geneID)
mi_genes = set(dep_mi_df.geneID)

In [87]:
len(mi_genes.intersection(sig_coxph_genes))

61

In [97]:
sig_deg_matrisome_df = (
    pd.read_csv(f"{dirs.analysis_dir}/{unified_dsets[i]}_sig_DESeq_results_xref_matrisome.tsv", sep='\t')
        .query("in_matrisome == True")
        .reset_index(drop=True)
)

In [99]:
sig_deg_genes = set(sig_deg_matrisome_df.geneID)

In [100]:
len(mi_genes.intersection(sig_deg_genes))

259